<a href="https://colab.research.google.com/github/thantham-ky/FaceDetection_AgeGender/blob/master/%E0%B8%AA%E0%B8%B3%E0%B9%80%E0%B8%99%E0%B8%B2%E0%B8%82%E0%B8%AD%E0%B8%87_gender_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import numpy as np

In [ ]:
x = np.load('/content/drive/My Drive/FaceDetection_AgeGender/imgdata.npy')
y = np.load('/content/drive/My Drive/FaceDetection_AgeGender/gender.npy')

In [ ]:
from keras.applications import Xception, MobileNetV2, ResNet50V2, InceptionV3, DenseNet201
from keras.models import  Model
from keras.layers import Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from keras.optimizers import RMSprop, Adam

from keras.utils import plot_model
from sklearn.model_selection import train_test_split

import pandas as pd

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True, 
                             rotation_range=10, 
                             width_shift_range=0.1, 
                             height_shift_range=0.1, 
                             zoom_range=0.1, 
                             rescale=0.1,
                             shear_range=0.1,
                             brightness_range=[0.1,0.2])

In [ ]:
base_model = DenseNet201(input_shape=x_train.shape[1:], include_top=False, pooling='avg')
x = base_model.output
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
gender_out = Dense(1, activation='sigmoid', name='gender_out')(x)
gender_model = Model(inputs=base_model.input, outputs=gender_out)

gender_model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

74842112/74836368 [==============================] - 1s 0us/step


In [ ]:
callbacks = [#EarlyStopping(monitor='loss', patience=10, verbose=True),
             ReduceLROnPlateau(monitor='loss', patience=3, verbose=True)]

In [ ]:
gender_history = gender_model.fit_generator(datagen.flow(x_train, y_train),
                                      steps_per_epoch=x_train.shape[0]//32,epochs=50,
                                      validation_data=datagen.flow(x_test, y_test),
                                      workers=-1, callbacks=callbacks)

Epoch 1/50
12/12 [==============================] - 17s 1s/step - loss: 0.5756 - accuracy: 0.7005 - val_loss: 1.0556 - val_accuracy: 0.5657
Epoch 2/50
12/12 [==============================] - 11s 931ms/step - loss: 0.3499 - accuracy: 0.8269 - val_loss: 1.1478 - val_accuracy: 0.5859
Epoch 3/50
12/12 [==============================] - 11s 933ms/step - loss: 0.2311 - accuracy: 0.8929 - val_loss: 1.1787 - val_accuracy: 0.5859
Epoch 4/50
12/12 [==============================] - 12s 977ms/step - loss: 0.1422 - accuracy: 0.9323 - val_loss: 1.1602 - val_accuracy: 0.5960
Epoch 5/50
12/12 [==============================] - 11s 937ms/step - loss: 0.0905 - accuracy: 0.9670 - val_loss: 1.2886 - val_accuracy: 0.6667
Epoch 6/50
12/12 [==============================] - 11s 950ms/step - loss: 0.0750 - accuracy: 0.9670 - val_loss: 0.6012 - val_accuracy: 0.7677
Epoch 7/50
12/12 [==============================] - 11s 944ms/step - loss: 0.0882 - accuracy: 0.9780 - val_loss: 0.7126 - val_accuracy: 0.7677
Ep

In [ ]:
gender_hist_df = pd.DataFrame(gender_history.history)
gender_hist_df.to_csv('/content/drive/My Drive/FaceDetection_AgeGender/history_dense201_gender.csv')
gender_model.save('/content/drive/My Drive/FaceDetection_AgeGender/model_dense201_gender.h5')